In [3]:
import requests
from ast import literal_eval
from oddf import odasdf

import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.http_constants as http_constants
import argparse
from pathlib import Path
import os
import json
from tqdm import tqdm

from ast import literal_eval
from os import listdir
from os.path import isfile, join
import concurrent.futures
from timer import timer

# azure-core==1.8.0
# azure-cosmos==4.0.0
# azure-storage-blob==12.1.0

In [4]:
r = requests.post("http://20.185.77.234:5000",
                    json={'ID': "10076" , 'FPS': '1', 'duration': '60', 'lang': "hindi",
                            'container': "athenaliveprod"})

In [14]:
with open('outputfile.json', 'wb') as outf:
    outf.write(r.content)

In [10]:
od_df = odasdf(literal_eval(r.text))

In [12]:
od_df


,area_percentage,count,label,score,frame,folder
0,12.50,1,wall,1.00,1,10076_
1,10.16,1,ceiling,1.00,1,10076_
2,4.39,1,floor,1.00,1,10076_
3,2.55,1,window,1.00,1,10076_
4,0.69,1,light,1.00,1,10076_
...,...,...,...,...,...,...
253,14.71,1,wall,1.00,52,10076_
254,6.93,1,ceiling,1.00,52,10076_
255,1.26,1,rug,1.00,52,10076_
256,0.72,1,floor,1.00,52,10076_


In [22]:
# Specify you cosmos account details

COSMOS_ACCOUNT_URI="https://cosmos-ml.documents.azure.com:443"
COSMOS_ACCOUNT_KEY="Xk2aRRmk45Ix6CJH72ZgzcbV0uQn4Ln2gYnAfdPY4gxi65X2odyA9BdIxlCWBkiWquodWSyHY7mFce1L5X9Nzg=="

client = cosmos_client.CosmosClient(url = COSMOS_ACCOUNT_URI, credential = COSMOS_ACCOUNT_KEY)

# Database and container name

database_name = 'pipeline'
database = client.get_database_client(database_name)
container_name = 'custom_od'
container = database.get_container_client(container_name)

In [23]:
#  Push util for COSMOS JSON
def push_files(files, basepath = None):
    for file in tqdm(files):
        with open(f'{basepath}/{file}.json', 'rb') as f:
            data = json.load(f)
            container.upsert_item(data)
            print('Upload completed for {">"*10} {file}')

In [24]:
# get list of files for upload 
basepath = f'./'
files = []
for f in listdir(f'{basepath}/'):
    if '.json' in f and os.path.getsize(f'{basepath}/{f}') != 0:
        files.append(f.split('.')[0])

In [26]:
push_files(files, basepath=".")

100%|██████████| 1/1 [00:00<00:00,  7.43it/s]

Upload completed for {">"*10} {file}
